<a href="https://colab.research.google.com/github/danm91/le_kingmakers/blob/seb_dl_models/notebooks/seb_distillBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistillBERT Model

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import os
# os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']='0'

from preproc_text import process_tweets
from preproc_abbv import abbreviations
from preproc_class import TextPreprocess

from tensorflow.keras.preprocessing.text import text_to_word_sequence

from sklearn.model_selection import train_test_split

import ktrain
from ktrain import text

## Import Data

In [ ]:
# importing data from hard disk
csv_path = os.path.join('/home/sbyhung/code/danm91/le_kingmakers/raw_data','training.1600000.processed.noemoticon.csv')
df = pd.read_csv(csv_path, header=None)
df = df.copy()

In [ ]:
# get sample for testing
sample_size = int(df.shape[0] * 0.0001)
data_sample = df.sample(sample_size, random_state=0)
data_sample.shape

(160, 6)

In [ ]:
# splitting data into X & y
X = data_sample.iloc[:, 5]
y = data_sample.iloc[:, 0]
display(X.shape)
display(y.shape)

(160,)

(160,)

In [ ]:
# changing positive values from 4 to 1
y = y.map({0: 0, 4:1})

In [ ]:
# cleaning with bespoke classes
X = X.apply(process_tweets)

In [ ]:
# splitting data train:test:val = 60:20:20
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=0, stratify=y_temp)
display(X_train.shape)
display(X_test.shape)
display(X_val.shape)
display(y_train.shape)
display(y_test.shape)
display(y_val.shape)

(96,)

(32,)

(32,)

(96,)

(32,)

(32,)

In [ ]:
# change pd series to array
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [ ]:
# check dimensions
display(X_train.shape)
display(X_test.shape)
display(X_val.shape)
display(y_train.shape)
display(y_test.shape)
display(y_val.shape)

(96,)

(32,)

(32,)

(96,)

(32,)

(32,)

## Processing Data

In [ ]:
#### Tokenise words

In [ ]:
y.unique()

array([0, 1])

## Define Model & Preprocess Data

In [ ]:
# model name & class name
model_name = 'distilbert-base-uncased'
class_names = y.unique()

In [ ]:
# transformer
trans = text.Transformer(model_name, maxlen=512, class_names=class_names)


In [ ]:
# preprocessing data
train_data = trans.preprocess_train(X_train, y_train)
test_data = trans.preprocess_test(X_test, y_test)
val_data = trans.preprocess_test(X_val, y_val)

preprocessing train...
language: en
train sequence lengths:
	mean : 7
	95percentile : 13
	99percentile : 15


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 7
	95percentile : 14
	99percentile : 15


preprocessing test...
language: en
test sequence lengths:
	mean : 6
	95percentile : 11
	99percentile : 13


## Instantiate Model & Find Learning Rate & Fit

In [ ]:
# instantiate model using get classifer
model = trans.get_classifier()

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [ ]:
# set learner
learner = ktrain.get_learner(model, train_data=train_data, val_data=val_data, batch_size=16)

In [ ]:
# fine best learning rate
learner.lr_find(show_plot=True, max_epochs=10)

simulating training for different learning rates... this may take a few moments...
Epoch 1/10
2/6 [=========>....................] - ETA: 20:54 - loss: 0.6854 - accuracy: 0.6562

In [ ]:
learner.fit_onecycle(le-4, 1)

## Evaluate Model Performance

In [ ]:
# Get performance of model
learner.validate(class_names=class_names)

In [ ]:
# View data points with bigget losses
learner.view_top_losses(n=5, preproc=trans)

## Predictions

In [ ]:
# define predictor function
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [ ]:
# state prediction
test_string = """some example string to be evaluated"""

In [ ]:
# get prediction result
predictor.predict(test_string)

## Saving Model

In [ ]:
#######  NEED TO ENTER THE FOLLOWING TEXT ######

In [ ]:
# run the text below NB. It is very large 00s of MB
predictor.save('name_of_model')